<a href="https://colab.research.google.com/github/Sushmitha-Katti/EVA-4/blob/master/Session12/S12_V2(Ran_the_complete_model)ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Object classification on Tiny Imagenet Data

In [6]:
!pip install --no-cache-dir git+https://github.com/albu/albumentations > /dev/null && echo "Albumentation library is successfully installed!"
!git clone https://github.com/Sushmitha-Katti/PyTNet 
import sys
sys.path.append("PyTNet")

  Running command git clone -q https://github.com/albu/albumentations /tmp/pip-req-build-hb1q_31p
Albumentation library is successfully installed!
fatal: destination path 'PyTNet' already exists and is not an empty directory.


## **Importing Required files**

In [0]:
import torch
import train_test_loader
import show_images
import train_test
import evaluate
from Albumentationtransform import AlbumentationTransforms
from GradCam import GradCAM,visualize_cam
from LR_Range_Test import LR_test
from tinyimagenet import TinyImageNetDataSet

In [0]:
import albumentations as A
import cv2
channel_means = (0.5, 0.5, 0.5)
channel_stdevs = (0.5, 0.5, 0.5)
train_transform = AlbumentationTransforms([       
                                                                
                                      A.HorizontalFlip(p = 0.7),
                                      A.PadIfNeeded(min_height=70, min_width=70, border_mode=4, value=None, mask_value=None, always_apply=False, p=1.0)
                                      A.GaussNoise(var_limit=(10.0, 50.0), mean=0, always_apply=False, p=0.5)
                                      A.RandomCrop(64, 64, always_apply=False, p=1.0)
                                      # A.transforms.ShiftScaleRotate()
                                      A.Normalize(mean=channel_means, std=channel_stdevs),
                                      A.Cutout(num_holes=1, max_h_size=32,max_w_size = 32,p=0.7) 
                                       ])
test_transform = AlbumentationTransforms([A.Normalize(mean=channel_means, std=channel_stdevs)])
train_dataset , test_dataset,classes = TinyImageNetDataSet(train_split = 70,test_transforms = test_transform,train_transforms = train_transform)

## **DataLoader**

In [0]:
trainloader, testloader = train_test_loader.load(train_dataset,test_dataset,batch_size=640)

## **Used Torchvisions resnet model, but not pre trained. Print the summary**

In [10]:
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = torchvision.models.resnet18(pretrained=False, num_classes=200).to(device)
summary(model, input_size=(3, 64, 64))


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           9,408
       BatchNorm2d-2           [-1, 64, 32, 32]             128
              ReLU-3           [-1, 64, 32, 32]               0
         MaxPool2d-4           [-1, 64, 16, 16]               0
            Conv2d-5           [-1, 64, 16, 16]          36,864
       BatchNorm2d-6           [-1, 64, 16, 16]             128
              ReLU-7           [-1, 64, 16, 16]               0
            Conv2d-8           [-1, 64, 16, 16]          36,864
       BatchNorm2d-9           [-1, 64, 16, 16]             128
             ReLU-10           [-1, 64, 16, 16]               0
       BasicBlock-11           [-1, 64, 16, 16]               0
           Conv2d-12           [-1, 64, 16, 16]          36,864
      BatchNorm2d-13           [-1, 64, 16, 16]             128
             ReLU-14           [-1, 64,

## **Initialise Optimiser,Scheduler, Criterion**

In [0]:
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import OneCycleLR

optimizer = optim.SGD(model.parameters(), lr=0.01,momentum=0.9 ) 
criterion = nn.CrossEntropyLoss()
scheduler = OneCycleLR(optimizer, max_lr = 0.2, total_steps=None, epochs=20, steps_per_epoch=1, pct_start=0.5, anneal_strategy='linear', cycle_momentum=True, base_momentum=0.85, max_momentum=0.95, div_factor=10.0,final_div_factor =1)


## **Train Model**

In [12]:

train_test.train_model(model,device,trainloader,testloader,optimizer,criterion,20,scheduler,batch_scheduler=False,best_acc = 0,path = "tinymodel.pt")

  0%|          | 0/121 [00:00<?, ?it/s]

EPOCH: 1 LR: 0.025


Loss=4.514608860015869 Batch_id=120 Accuracy=3.74: 100%|██████████| 121/121 [03:30<00:00,  1.74s/it]
  0%|          | 0/121 [00:00<?, ?it/s]


Test set: Average loss: 0.0073, Accuracy: 2254/33000 (6.83%)

accuracy increased, Saving model....
EPOCH: 2 LR: 0.045454545454545456


Loss=3.735003709793091 Batch_id=120 Accuracy=11.54: 100%|██████████| 121/121 [03:29<00:00,  1.74s/it]
  0%|          | 0/121 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0064, Accuracy: 11875/33000 (35.98%)

accuracy increased, Saving model....
EPOCH: 23 LR: 0.04375000000000001


Loss=0.07350314408540726 Batch_id=120 Accuracy=99.02: 100%|██████████| 121/121 [03:27<00:00,  1.72s/it]
  0%|          | 0/121 [00:00<?, ?it/s]


Test set: Average loss: 0.0064, Accuracy: 12007/33000 (36.38%)

accuracy increased, Saving model....
EPOCH: 24 LR: 0.024999999999999994


Loss=0.04106489196419716 Batch_id=120 Accuracy=99.20: 100%|██████████| 121/121 [03:27<00:00,  1.71s/it]
100%|██████████| 52/52 [00:08<00:00,  5.99it/s]


Test set: Average loss: 0.0064, Accuracy: 11995/33000 (36.35%)

